In [2]:
"Fastai Language Model Databunch modified to work with music"

'Fastai Language Model Databunch modified to work with music'

In [3]:
# from fastai.basics import *
# from fastai.basic_data import DataBunch
# from fastai.text.data import LMLabelList

from musicautobot.music_transformer.transform import *
from musicautobot.vocab import MusicVocab

In [4]:
from fastai.text.data import *
from fastai.basics import *
from fastai.text.all import *

In [5]:
path = untar_data(URLs.WIKITEXT_TINY)

In [6]:
df_train = pd.read_csv(path/'train.csv', header=None)
df_valid = pd.read_csv(path/'test.csv', header=None)
df_all = pd.concat([df_train, df_valid])

In [7]:
splits = [list(range_of(df_train)), list(range(len(df_train), len(df_all)))]
tfms = [attrgetter("text"), Tokenizer.from_df(0), Numericalize()]
dsets = Datasets(df_all, [tfms], splits=splits)

/home/ashaw/anaconda3/envs/musicautobot/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [8]:
dls = LMDataLoader(dsets, bs=12, seq_len=24)

In [9]:
dls.items[0]

TensorText([  2,  17, 444,  ...,  52,   8, 872])

In [10]:
cs = Chunks(dls.items, dls.lens)

## HuggingFace

In [11]:
from transformers import AutoTokenizer, AutoModel

In [12]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [13]:
inputs = tokenizer("Hello world!", return_tensors="pt")

In [14]:
inputs

{'input_ids': tensor([[ 101, 7592, 2088,  999,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [15]:
# bs,sl = 104,512
# dls = dsets.dataloaders(bs=bs, seq_len=sl)

## Music Model

In [16]:
base_path = Path('../../data')

# Location of your midi files
midi_path = base_path/'midi/examples'
midi_path.mkdir(parents=True, exist_ok=True)

# Location to save dataset
data_path = base_path/'numpy'
data_path.mkdir(parents=True, exist_ok=True)

data_save_name = 'musicitem_data_save.pkl'

In [17]:
midi_files = get_files(midi_path, '.mid', recurse=True); len(midi_files)

31

In [18]:
vocab = MusicVocab.create()

In [19]:
splits = RandomSplitter(seed=42)(range(len(midi_files)))

In [20]:

class Midi2ItemProcessor(Transform):
    "Skips midi preprocessing step. And encodes midi files to MusicItems"
    def __init__(self,vocab):
        self.vocab = vocab
    
    def encodes(self, o):
        return MusicItem.from_file(o, vocab=self.vocab)
    

In [21]:
tfms = [Midi2ItemProcessor(vocab)]
dsets = Datasets(midi_files, [tfms], splits=splits)

In [22]:
music_items = [MusicItem.from_file(f, vocab=vocab) for f in midi_files]

In [ ]:
# class MusicTensor():
#     def __init__(self, data, vocab, transpose=0):
#         self.data = data
#         self.vocab = vocab
#     @classmethod
#     def from_item(cls, mi):
#         return cls(np.stack([mi.data, mi.position]).T, mi.vocab)
    
#     def __getitem__(self,i):
#         return self.data[i]
#         if isinstance(i,slice): return retain_type(self.getslice(i), old=self.chunks[0])
#         di,idx = self.doc_idx(i)
#         return retain_type(self.chunks[di][idx], old=self.chunks[0])

In [24]:
import random

In [32]:
random.randint(-6, 6)

-4

In [39]:
random.random()

0.7875462847226555

In [ ]:
def mi2tensor(mi): return np.stack([mi.data, mi.position]).T

def rand_transpose(mi, steps=6, p=0.5):
    if random.random() >= p: return mi
    val = random.randint(-steps, steps)
    return mi.transpose(val)

class MusicItemDataLoader(LMDataLoader):
    def make_chunks(self): 
        items = [mi2tensor(rand_transpose(mi)) for mi in self.items]
        self.chunks = Chunks(items, self.lens)
    def shuffle_fn(self,idxs):
        self.items.shuffle()
        self.make_chunks()
        return idxs

In [ ]:
class Chunks:
    "Slice and int indexing into a list of lists"
    def __init__(self, chunks, lens=None):
        self.chunks = chunks
        self.lens = L(map(len,self.chunks) if lens is None else lens)
        self.cumlens = np.cumsum(0+self.lens)
        self.totlen = self.cumlens[-1]

    def __getitem__(self,i):
        if isinstance(i,slice): return retain_type(self.getslice(i), old=self.chunks[0])
        di,idx = self.doc_idx(i)
        return retain_type(self.chunks[di][idx], old=self.chunks[0])

    def getslice(self, i):
        st_d,st_i = self.doc_idx(ifnone(i.start,0))
        en_d,en_i = self.doc_idx(ifnone(i.stop,self.totlen+1))
        res = [self.chunks[st_d][st_i:(en_i if st_d==en_d else sys.maxsize)]]
        for b in range(st_d+1,en_d): res.append(self.chunks[b])
        if st_d!=en_d and en_d<len(self.chunks): res.append(self.chunks[en_d][:en_i])
        return concat(*res)

    def doc_idx(self, i):
        if i<0: i=self.totlen+i # count from end
        docidx = np.searchsorted(self.cumlens, i+1)-1
        cl = self.cumlens[docidx]
        return docidx,i-cl

In [ ]:
class ReindexCollection(GetAttr, IterLen):
    "Reindexes collection `coll` with indices `idxs` and optional LRU cache of size `cache`"
    _default='coll'
    def __init__(self, coll, idxs=None, cache=None, tfm=noop):
        if idxs is None: idxs = L.range(coll)
        store_attr()
        if cache is not None: self._get = functools.lru_cache(maxsize=cache)(self._get)

    def _get(self, i): return self.tfm(self.coll[i])
    def __getitem__(self, i): return self._get(self.idxs[i])
    def __len__(self): return len(self.coll)
    def reindex(self, idxs): self.idxs = idxs
    def shuffle(self): random.shuffle(self.idxs)
    def cache_clear(self): self._get.cache_clear()
    def __getstate__(self): return {'coll': self.coll, 'idxs': self.idxs, 'cache': self.cache, 'tfm': self.tfm}
    def __setstate__(self, s): self.coll,self.idxs,self.cache,self.tfm = s['coll'],s['idxs'],s['cache'],s['tfm']

In [6]:
@delegates()
class LMDataLoader(TfmdDL):
    "A `DataLoader` suitable for language modeling"
    def __init__(self, dataset, lens=None, cache=2, bs=64, seq_len=72, num_workers=0, **kwargs):
        self.items = ReindexCollection(dataset, cache=cache, tfm=_maybe_first)
        self.seq_len = seq_len
        if lens is None: lens = _get_lengths(dataset)
        if lens is None: lens = [len(o) for o in self.items]
        self.lens = ReindexCollection(lens, idxs=self.items.idxs)
        # The "-1" is to allow for final label, we throw away the end that's less than bs
        corpus = round_multiple(sum(lens)-1, bs, round_down=True)
        self.bl = corpus//bs #bl stands for batch length
        self.n_batches = self.bl//(seq_len) + int(self.bl%seq_len!=0)
        self.last_len = self.bl - (self.n_batches-1)*seq_len
        self.make_chunks()
        super().__init__(dataset=dataset, bs=bs, num_workers=num_workers, **kwargs)
        self.n = self.n_batches*bs

    def make_chunks(self): self.chunks = Chunks(self.items, self.lens)
    def shuffle_fn(self,idxs):
        self.items.shuffle()
        self.make_chunks()
        return idxs

    def create_item(self, seq):
        if seq>=self.n: raise IndexError
        sl = self.last_len if seq//self.bs==self.n_batches-1 else self.seq_len
        st = (seq%self.bs)*self.bl + (seq//self.bs)*self.seq_len
        txt = self.chunks[st : st+sl+1]
        return LMTensorText(txt[:-1]),txt[1:]

    @delegates(TfmdDL.new)
    def new(self, dataset=None, seq_len=None, **kwargs):
        lens = self.lens.coll if dataset is None else None
        seq_len = self.seq_len if seq_len is None else seq_len
        return super().new(dataset=dataset, lens=lens, seq_len=seq_len, **kwargs)

In [37]:
music_items[0].data

array([  0,   1,  69, 141,  66, 141,  62, 141,   8, 141,  69, 141,   8,
       141,  61, 138,   8, 138,  64, 139,   8, 139,  69, 139,   8, 139,
        64, 140,   8, 138,  64, 139,  61, 139,  57, 139,   8, 141,  59,
       139,  56, 139,  52, 139,   8, 139,  59, 141,  56, 141,  52, 141,
         8, 139,  61, 139,   8, 139,  61, 138,   8, 138,  64, 139,   8,
       139,  73, 140,   8, 140,  69, 141,  61, 139,  57, 139,  54, 139,
         8, 141,  69, 139,  66, 139,  62, 139,   8, 139,  69, 141,  66,
       141,  62, 141,   8, 139,  69, 139,   8, 139,  73, 139,   8, 139,
        74, 139,   8, 139,  73, 139,   8, 139,  69, 139,  64, 139,  61,
       139,  57, 139,   8, 141,  59, 139,  56, 139,  52, 139,   8, 139,
        59, 141,  56, 141,  52, 141,   8, 139,  69, 139,   8, 139,  73,
       139,   8, 139,  71, 139,   8, 139,  71, 139,   8, 139,  69, 139])

In [39]:
mi = music_items[0]

In [51]:
np.stack([mi.data, mi.position]).T.shape

(156, 2)

In [52]:
tensor_items = [np.stack([mi.data, mi.position]).T for mi in music_items]

In [53]:
lens = [len(mi) for mi in music_items]

In [54]:
cs = Chunks(tensor_items, lens)

In [49]:
tensor_items[0][]

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  4,  4,  4,  8,  8,
         8,  8,  9,  9,  9,  9, 11, 11, 11, 11, 13, 13, 13, 13, 14, 14,
        14, 14, 14, 14, 14, 14, 18, 18, 18, 18, 18, 18, 18, 18, 20, 20,
        20, 20, 20, 20, 20, 20, 22, 22, 22, 22, 24, 24, 24, 24, 25, 25,
        25, 25, 27, 27, 27, 27, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30,
        34, 34, 34, 34, 34, 34, 34, 34, 36, 36, 36, 36, 36, 36, 36, 36,
        38, 38, 38, 38, 40, 40, 40, 40, 42, 42, 42, 42, 44, 44, 44, 44,
        46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 50, 50, 50, 50, 50, 50,
        50, 50, 52, 52, 52, 52, 52, 52, 52, 52, 54, 54, 54, 54, 56, 56,
        56, 56, 58, 58, 58, 58, 60, 60, 60, 60, 62, 62]])

In [56]:
cs[1:100]

array([[  1,   0],
       [ 69,   0],
       [141,   0],
       [ 66,   0],
       [141,   0],
       [ 62,   0],
       [141,   0],
       [  8,   0],
       [141,   0],
       [ 69,   4],
       [141,   4],
       [  8,   4],
       [141,   4],
       [ 61,   8],
       [138,   8],
       [  8,   8],
       [138,   8],
       [ 64,   9],
       [139,   9],
       [  8,   9],
       [139,   9],
       [ 69,  11],
       [139,  11],
       [  8,  11],
       [139,  11],
       [ 64,  13],
       [140,  13],
       [  8,  13],
       [138,  13],
       [ 64,  14],
       [139,  14],
       [ 61,  14],
       [139,  14],
       [ 57,  14],
       [139,  14],
       [  8,  14],
       [141,  14],
       [ 59,  18],
       [139,  18],
       [ 56,  18],
       [139,  18],
       [ 52,  18],
       [139,  18],
       [  8,  18],
       [139,  18],
       [ 59,  20],
       [141,  20],
       [ 56,  20],
       [141,  20],
       [ 52,  20],
       [141,  20],
       [  8,  20],
       [139,

In [ ]:
class MusicItemProcessor(PreProcessor):
    "`PreProcessor` that transforms numpy files to indexes for training"
    def process_one(self,item):
        item = MusicItem.from_npenc(item, vocab=self.vocab)
        return item.to_idx()
    
    def process(self, ds):
        self.vocab = ds.vocab
        super().process(ds)
        
class OpenNPFileProcessor(PreProcessor):
    "`PreProcessor` that opens the filenames and read the texts."
    def process_one(self,item):
        return np.load(item, allow_pickle=True) if isinstance(item, Path) else item

class Midi2ItemProcessor(PreProcessor):
    "Skips midi preprocessing step. And encodes midi files to MusicItems"
    def process_one(self,item):
        item = MusicItem.from_file(item, vocab=self.vocab)
        return item.to_idx()
    
    def process(self, ds):
        self.vocab = ds.vocab
        super().process(ds)
    

In [18]:
??Midi2ItemProcessor

Object `Midi2ItemProcessor` not found.


In [ ]:
# processors = [Midi2ItemProcessor()]
# data = MusicDataBunch.from_files(midi_files, data_path, processors=processors, bs=2, bptt=12)
# data.save(data_save_name)

In [28]:
class Chunks:
    "Slice and int indexing into a list of lists"
    def __init__(self, chunks, lens=None):
        self.chunks = chunks
        self.lens = L(map(len,self.chunks) if lens is None else lens)
        self.cumlens = np.cumsum(0+self.lens)
        self.totlen = self.cumlens[-1]

    def __getitem__(self,i):
        if isinstance(i,slice): return retain_type(self.getslice(i), old=self.chunks[0])
        di,idx = self.doc_idx(i)
        return retain_type(self.chunks[di][idx], old=self.chunks[0])

    def getslice(self, i):
        st_d,st_i = self.doc_idx(ifnone(i.start,0))
        en_d,en_i = self.doc_idx(ifnone(i.stop,self.totlen+1))
        res = [self.chunks[st_d][st_i:(en_i if st_d==en_d else sys.maxsize)]]
        for b in range(st_d+1,en_d): res.append(self.chunks[b])
        if st_d!=en_d and en_d<len(self.chunks): res.append(self.chunks[en_d][:en_i])
        return concat(*res)

    def doc_idx(self, i):
        if i<0: i=self.totlen+i # count from end
        docidx = np.searchsorted(self.cumlens, i+1)-1
        cl = self.cumlens[docidx]
        return docidx,i-cl


In [27]:
??Chunks

In [26]:
cs

In [22]:
dir(cs)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'chunks',
 'cumlens',
 'doc_idx',
 'getslice',
 'lens',
 'totlen']

In [9]:
??Chunks

In [39]:
dls.show_batch(max_n=3)

,text,text_
0,"xxbos = 2013 – 14 xxmaj york xxmaj city xxup f.c . season = \n▁\n▁ xxmaj the 2013 – 14 season was the xxunk season of competitive association football and 77th season in the xxmaj football xxmaj league played by xxmaj york xxmaj city xxmaj football xxmaj club , a professional football club based in xxmaj york , xxmaj north xxmaj yorkshire , xxmaj england . xxmaj their 17th - place finish in 2012 – 13 meant it was their second consecutive season in xxmaj league xxmaj two . xxmaj the season ran from 1 xxmaj july 2013 to 30 xxmaj june 2014 . \n▁ xxmaj nigel xxmaj worthington , starting his first full season as xxmaj york manager , made eight permanent summer signings . xxmaj by the turn of the year xxmaj york were only above the relegation zone on goal difference , before a xxunk unbeaten run","= 2013 – 14 xxmaj york xxmaj city xxup f.c . season = \n▁\n▁ xxmaj the 2013 – 14 season was the xxunk season of competitive association football and 77th season in the xxmaj football xxmaj league played by xxmaj york xxmaj city xxmaj football xxmaj club , a professional football club based in xxmaj york , xxmaj north xxmaj yorkshire , xxmaj england . xxmaj their 17th - place finish in 2012 – 13 meant it was their second consecutive season in xxmaj league xxmaj two . xxmaj the season ran from 1 xxmaj july 2013 to 30 xxmaj june 2014 . \n▁ xxmaj nigel xxmaj worthington , starting his first full season as xxmaj york manager , made eight permanent summer signings . xxmaj by the turn of the year xxmaj york were only above the relegation zone on goal difference , before a xxunk unbeaten run saw"
1,". ) \n▁ xxmaj pulaski 's apprehension at using the transporter was evident in "" xxmaj the xxunk xxmaj man "" , where xxmaj dr . xxunk ( xxmaj suzie xxmaj plakson ) went with the away team instead of xxmaj pulaski , as it required her to beam over to a transport vessel . xxmaj however , the transporter would later save xxmaj pulaski 's life in "" xxmaj unnatural xxmaj selection "" , after she was infected with a disease from the planet xxunk xxup iv that accelerated her aging process . xxmaj she uses the xxunk to remove the infection and is returned to health . \n▁ xxmaj she demonstrates her medical expertise on several occasions . xxmaj in "" xxmaj time xxunk "" , xxmaj pulaski discovers that the duplicate xxmaj captain xxmaj picard is out of sync in time and will slowly improve until he",") \n▁ xxmaj pulaski 's apprehension at using the transporter was evident in "" xxmaj the xxunk xxmaj man "" , where xxmaj dr . xxunk ( xxmaj suzie xxmaj plakson ) went with the away team instead of xxmaj pulaski , as it required her to beam over to a transport vessel . xxmaj however , the transporter would later save xxmaj pulaski 's life in "" xxmaj unnatural xxmaj selection "" , after she was infected with a disease from the planet xxunk xxup iv that accelerated her aging process . xxmaj she uses the xxunk to remove the infection and is returned to health . \n▁ xxmaj she demonstrates her medical expertise on several occasions . xxmaj in "" xxmaj time xxunk "" , xxmaj pulaski discovers that the duplicate xxmaj captain xxmaj picard is out of sync in time and will slowly improve until he returns"
2,"xxmaj his xxmaj prime xxunk lasted barely a year , marred by problems and difficulties . xxmaj for the remainder of his life and without his wife , as xxmaj queen xxmaj victoria xxunk it , "" to hold him back "" , he became more and more eccentric and controversial in his decisions . xxmaj his final years were xxunk by ill health and a self - enforced seclusion in xxmaj scotland . xxmaj he died in 1929 . \n▁ xxmaj before their marriage and his full - time entry into politics , xxmaj rosebery 's future wife had written with extraordinary xxunk and ambition to him : "" i work only to help you , if you are xxmaj prime xxmaj minister , let me imitate xxmaj montagu xxunk . "" xxunk had been xxmaj disraeli 's influential private secretary on whom he had relied . 

In [40]:
xb,yb = dls.one_batch()

In [57]:
??ReindexCollection

In [4]:
class MusicDataBunch(DataBunch):
    "Create a `TextDataBunch` suitable for training a language model."
    @classmethod
    def create(cls, train_ds, valid_ds, test_ds=None, path:PathOrStr='.', no_check:bool=False, bs=64, val_bs:int=None, 
               num_workers:int=0, device:torch.device=None, collate_fn:Callable=data_collate, 
               dl_tfms:Optional[Collection[Callable]]=None, bptt:int=70,
               preloader_cls=None, shuffle_dl=False, transpose_range=(0,12), **kwargs) -> DataBunch:
        "Create a `TextDataBunch` in `path` from the `datasets` for language modelling."
        datasets = cls._init_ds(train_ds, valid_ds, test_ds)
        preloader_cls = MusicPreloader if preloader_cls is None else preloader_cls
        val_bs = ifnone(val_bs, bs)
        datasets = [preloader_cls(ds, shuffle=(i==0), bs=(bs if i==0 else val_bs), bptt=bptt, transpose_range=transpose_range, **kwargs) 
                    for i,ds in enumerate(datasets)]
        val_bs = bs
        dl_tfms = [partially_apply_vocab(tfm, train_ds.vocab) for tfm in listify(dl_tfms)]
        dls = [DataLoader(d, b, shuffle=shuffle_dl) for d,b in zip(datasets, (bs,val_bs,val_bs,val_bs)) if d is not None]
        return cls(*dls, path=path, device=device, dl_tfms=dl_tfms, collate_fn=collate_fn, no_check=no_check)
    
    @classmethod    
    def from_folder(cls, path:PathOrStr, extensions='.npy', **kwargs):
        files = get_files(path, extensions=extensions, recurse=True);
        return cls.from_files(files, path, **kwargs)
    
    @classmethod
    def from_files(cls, files, path, processors=None, split_pct=0.1, 
                   vocab=None, list_cls=None, **kwargs):
        if vocab is None: vocab = MusicVocab.create()
        if list_cls is None: list_cls = MusicItemList
        src = (list_cls(items=files, path=path, processor=processors, vocab=vocab)
                .split_by_rand_pct(split_pct, seed=6)
                .label_const(label_cls=LMLabelList))
        return src.databunch(**kwargs)

    @classmethod
    def empty(cls, path, **kwargs):
        vocab = MusicVocab.create()
        src = MusicItemList([], path=path, vocab=vocab, ignore_empty=True).split_none()
        return src.label_const(label_cls=LMLabelList).databunch()
        
     

NameError: name 'DataBunch' is not defined

In [38]:
   
def partially_apply_vocab(tfm, vocab):
    if 'vocab' in inspect.getfullargspec(tfm).args:
        return partial(tfm, vocab=vocab)
    return tfm
    
class MusicItemList(ItemList):
    _bunch = MusicDataBunch
    
    def __init__(self, items:Iterator, vocab:MusicVocab=None, **kwargs):
        super().__init__(items, **kwargs)
        self.vocab = vocab
        self.copy_new += ['vocab']
    
    def get(self, i):
        o = super().get(i)
        if is_pos_enc(o): 
            return MusicItem.from_idx(o, self.vocab)
        return MusicItem(o, self.vocab)

NameError: name 'ItemList' is not defined

In [ ]:
def is_pos_enc(idxenc):
    if len(idxenc.shape) == 2 and idxenc.shape[0] == 2: return True
    return idxenc.dtype == np.object and idxenc.shape == (2,)

In [ ]:
class MusicItemProcessor(PreProcessor):
    "`PreProcessor` that transforms numpy files to indexes for training"
    def process_one(self,item):
        item = MusicItem.from_npenc(item, vocab=self.vocab)
        return item.to_idx()
    
    def process(self, ds):
        self.vocab = ds.vocab
        super().process(ds)
        
class OpenNPFileProcessor(PreProcessor):
    "`PreProcessor` that opens the filenames and read the texts."
    def process_one(self,item):
        return np.load(item, allow_pickle=True) if isinstance(item, Path) else item

class Midi2ItemProcessor(PreProcessor):
    "Skips midi preprocessing step. And encodes midi files to MusicItems"
    def process_one(self,item):
        item = MusicItem.from_file(item, vocab=self.vocab)
        return item.to_idx()
    
    def process(self, ds):
        self.vocab = ds.vocab
        super().process(ds)
    

In [ ]:
## For npenc dataset
class MusicPreloader(Callback):
    "Transforms the tokens in `dataset` to a stream of contiguous batches for language modelling."
    
    class CircularIndex():
        "Handles shuffle, direction of indexing, wraps around to head tail in the ragged array as needed"
        def __init__(self, length:int, forward:bool): self.idx, self.forward = np.arange(length), forward
        def __getitem__(self, i): 
            return self.idx[ i%len(self.idx) if self.forward else len(self.idx)-1-i%len(self.idx)]
        def __len__(self) -> int: return len(self.idx)
        def shuffle(self): np.random.shuffle(self.idx)

    def __init__(self, dataset:LabelList, lengths:Collection[int]=None, bs:int=32, bptt:int=70, backwards:bool=False, 
                 shuffle:bool=False, y_offset:int=1, 
                 transpose_range=None, transpose_p=0.5,
                 encode_position=True,
                 **kwargs):
        self.dataset,self.bs,self.bptt,self.shuffle,self.backwards,self.lengths = dataset,bs,bptt,shuffle,backwards,lengths
        self.vocab = self.dataset.vocab
        self.bs *= num_distrib() or 1
        self.totalToks,self.ite_len,self.idx = int(0),None,None
        self.y_offset = y_offset
        
        self.transpose_range,self.transpose_p = transpose_range,transpose_p
        self.encode_position = encode_position
        self.bptt_len = self.bptt
        
        self.allocate_buffers() # needed for valid_dl on distributed training - otherwise doesn't get initialized on first epoch

    def __len__(self): 
        if self.ite_len is None:
            if self.lengths is None: self.lengths = np.array([len(item) for item in self.dataset.x])
            self.totalToks = self.lengths.sum()
            self.ite_len   = self.bs*int( math.ceil( self.totalToks/(self.bptt*self.bs) )) if self.item is None else 1
        return self.ite_len

    def __getattr__(self,k:str)->Any: return getattr(self.dataset, k)
   
    def allocate_buffers(self):
        "Create the ragged array that will be filled when we ask for items."
        if self.ite_len is None: len(self)
        self.idx   = MusicPreloader.CircularIndex(len(self.dataset.x), not self.backwards)
        
        # batch shape = (bs, bptt, 2 - [index, pos]) if encode_position. Else - (bs, bptt)
        buffer_len = (2,) if self.encode_position else ()
        self.batch = np.zeros((self.bs, self.bptt+self.y_offset) + buffer_len, dtype=np.int64)
        self.batch_x, self.batch_y = self.batch[:,0:self.bptt], self.batch[:,self.y_offset:self.bptt+self.y_offset] 
        #ro: index of the text we're at inside our datasets for the various batches
        self.ro    = np.zeros(self.bs, dtype=np.int64)
        #ri: index of the token we're at inside our current text for the various batches
        self.ri    = np.zeros(self.bs, dtype=np.int)
        
        # allocate random transpose values. Need to allocate this before hand.
        self.transpose_values = self.get_random_transpose_values()
        
    def get_random_transpose_values(self):
        if self.transpose_range is None: return None
        n = len(self.dataset)
        rt_arr = torch.randint(*self.transpose_range, (n,))-self.transpose_range[1]//2
        mask = torch.rand(rt_arr.shape) > self.transpose_p
        rt_arr[mask] = 0
        return rt_arr

    def on_epoch_begin(self, **kwargs):
        if self.idx is None: self.allocate_buffers()
        elif self.shuffle:   
            self.ite_len = None
            self.idx.shuffle()
            self.transpose_values = self.get_random_transpose_values()
            self.bptt_len = self.bptt
        self.idx.forward = not self.backwards 

        step = self.totalToks / self.bs
        ln_rag, countTokens, i_rag = 0, 0, -1
        for i in range(0,self.bs):
            #Compute the initial values for ro and ri 
            while ln_rag + countTokens <= int(step * i):
                countTokens += ln_rag
                i_rag       += 1
                ln_rag       = self.lengths[self.idx[i_rag]]
            self.ro[i] = i_rag
            self.ri[i] = ( ln_rag - int(step * i - countTokens) ) if self.backwards else int(step * i - countTokens)
        
    #Training dl gets on_epoch_begin called, val_dl, on_epoch_end
    def on_epoch_end(self, **kwargs): self.on_epoch_begin()

    def __getitem__(self, k:int):
        j = k % self.bs
        if j==0:
            if self.item is not None: return self.dataset[0]
            if self.idx is None: self.on_epoch_begin()
                
        self.ro[j],self.ri[j] = self.fill_row(not self.backwards, self.dataset.x, self.idx, self.batch[j][:self.bptt_len+self.y_offset], 
                                              self.ro[j], self.ri[j], overlap=1, lengths=self.lengths)
        return self.batch_x[j][:self.bptt_len], self.batch_y[j][:self.bptt_len]

    def fill_row(self, forward, items, idx, row, ro, ri, overlap, lengths):
        "Fill the row with tokens from the ragged array. --OBS-- overlap != 1 has not been implemented"
        ibuf = n = 0 
        ro  -= 1
        while ibuf < row.shape[0]:  
            ro   += 1 
            ix    = idx[ro]
            
            item = items[ix]
            if self.transpose_values is not None: 
                item = item.transpose(self.transpose_values[ix].item())
                
            if self.encode_position:
                # Positions are colomn stacked with indexes. This makes it easier to keep in sync
                rag = np.stack([item.data, item.position], axis=1)
            else:
                rag = item.data
                
            if forward:
                ri = 0 if ibuf else ri
                n  = min(lengths[ix] - ri, row.shape[0] - ibuf)
                row[ibuf:ibuf+n] = rag[ri:ri+n]
            else:    
                ri = lengths[ix] if ibuf else ri
                n  = min(ri, row.size - ibuf) 
                row[ibuf:ibuf+n] = rag[ri-n:ri][::-1]
            ibuf += n
        return ro, ri + ((n-overlap) if forward else -(n-overlap))

def batch_position_tfm(b):
    "Batch transform for training with positional encoding"
    x,y = b
    x = {
        'x': x[...,0],
        'pos': x[...,1]
    }
    return x, y[...,0]
    